In [5]:
from collections import defaultdict
from heapq import heappop, heappush, heapify
from PIL import Image


class Node:
    def __init__(self, freq, pixel=None):
        self.freq = freq
        self.pixel = pixel
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq


def build_huffman_tree(frequencies):
    priority_queue = [Node(freq, pixel) for pixel, freq in frequencies.items()]
    heapify(priority_queue)

    while len(priority_queue) > 1:
        left = heappop(priority_queue)
        right = heappop(priority_queue)

        merged = Node(left.freq + right.freq)
        merged.left = left
        merged.right = right

        heappush(priority_queue, merged)

    return priority_queue[0]


def build_huffman_codes(root, current_code="", huffman_codes={}):
    if root is None:
        return

    if root.pixel is not None:
        huffman_codes[root.pixel] = current_code

    build_huffman_codes(root.left, current_code + "0", huffman_codes)
    build_huffman_codes(root.right, current_code + "1", huffman_codes)

    return huffman_codes


def encode_image(img):
    width, height = img.size
    pixels = list(img.getdata())

    frequencies = defaultdict(int)
    for pixel in pixels:
        frequencies[pixel] += 1

    root = build_huffman_tree(frequencies)
    huffman_codes = build_huffman_codes(root)

    encoded_pixels = "".join(huffman_codes[pixel] for pixel in pixels)

    return encoded_pixels, root, width, height


def decode_image(encoded_pixels, root, width, height):
    decoded_pixels = []
    current_node = root

    for bit in encoded_pixels:
        if bit == "0":
            current_node = current_node.left
        else:
            current_node = current_node.right

        if current_node.pixel is not None:
            decoded_pixels.append(current_node.pixel)
            current_node = root

    return decoded_pixels


# Example usage
input_image_path = "Dog.jpg"  # Replace with your input image file path
output_image_path = "output_image.jpg"

# Open the image
image = Image.open(input_image_path)
encoded_data, huffman_tree, width, height = encode_image(image)

# Decode the data
decoded_data = decode_image(encoded_data, huffman_tree, width, height)

# Create a new image and save
decoded_image = Image.new("RGB", (width, height))
decoded_image.putdata(decoded_data)
decoded_image.save(output_image_path)